<a href="https://colab.research.google.com/github/aaronjoseph/siim-covid19-detection/blob/master/EDA_2_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -q efficientnet >> /dev/null
!sudo pip install --upgrade --force-reinstall --no-deps "kaggle>=1.5.12"
!apt-get install -y fuse-zip
!pip install pydicom
!pip install kaggle
!pip install tensorflow-addons

Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fuse-zip is already the newest version (0.4.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [24]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"aaronjoseph","key":"1fa60446a1bf44f239d2ca4ddda1d9f4"}

api_key = "9d9646b4fe5cfedcb7609d8f0773846848ecbcae"
!wandb login $api_key

import json
import os
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json

# Downloading the Data
!kaggle competitions download -c siim-covid19-detection -p /content/Input

# Using fuse-zip
input_dir = "/content/Input/siim"
!mkdir {input_dir}
!fuse-zip /content/Input/siim-covid19-detection.zip {input_dir}

/bin/bash: wandb: command not found
100% 83.2G/83.2G [18:10<00:00, 68.1MB/s]
100% 83.2G/83.2G [18:10<00:00, 81.9MB/s]


In [20]:
import pandas as pd, numpy as np, random, os, shutil, matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf, tensorflow.keras.backend as K, efficientnet.tfkeras as efn, tensorflow_addons as tfa
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
print('tf:',tf.__version__)

tf: 2.5.0


In [21]:
# VERBOSE = 0, Silent, 1 interactive, 2 commit
VERBOSE = 0
DISPLAY_PLOT = True
DEVICE = "TPU"

# Use different seed for different stratified KFOLD
SEED = 42

# No of Folds
FOLDS = 5

# Image Size Load
IMG_SIZE = [[512,512]] * FOLDS

#BATCH SIZE & EPOCHS
BATCH_SIZES = [16]*FOLDS
EPOCHS = [12]*FOLDS

# WHICH EFFICIENTNET TO USE
EFF_NET = [7]*FOLDS

# AUGMENTATIONS
AUGMENT = True
TRANSFORM = True

# TRANSFORMATIONS
ROT_ = 0.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

# DROPOUT
PROBABILITY = 0.75
CT = 8
SZ = 0.08

# BRI, CONSTRAST
sat = (0.7,1.3)
cont = (0.8, 1.2)
bri = 0.1

# WEIGHTS FOR FOLD MODELS WHEN PREDICTING TEST
WGTS = [1/FOLDS]*FOLDS

# TEST TIME AUGMENTATION STEPS
TTA = 1

In [22]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
    tf.random.set_seed(SEED)
seeding(SEED)

In [25]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

connecting to TPU...
Running on TPU  grpc://10.115.120.234:8470
initializing  TPU ...


INFO:tensorflow:Initializing the TPU system: grpc://10.115.120.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.115.120.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TPU initialized
REPLICAS: 8
